# Описание задания
Загрузите данные и посчитайте модели линейной регрессии для 50 зданий по ансамблю регрессионных моделей:
* в первой модели весь оптимальный набор метеорологических данных,
* во второй - дни недели и праздники,
* в третьей - недели года,
* в четвертой - месяцы.

Финальное значение показателя рассчитайте как взвешенное арифметическое среднее показателей всех моделей, взяв веса для первой и второй модели как 3/8, а для третьей и четвертой - как 1/8.

Загрузите данные решения, посчитайте значение энергопотребления для требуемых дат для тех зданий, которые посчитаны в модели, и выгрузите результат в виде CSV-файла (submission.csv).

Данные:
* http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/test.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/weather_test.csv.gz

Итоговый файл с кодом (.py или .ipynb) выложите в github с портфолио.
<a id='task_definition'></a>
## 1. Определение задания

In [1]:
NUMBER_OF_BUILDINGS_FOR_MODELLING = 50

WEEKDAY_FEATURES = ['is_wd' + str(i) for i in range(0, 7)]
WEEK_FEATURES = ['is_week' + str(i) for i in range(1, 54)]
MONTH_FEATURES = ['is_month' + str(i) for i in range(1, 13)]

FEATURE_TYPES = {
    1: #в первой модели весь оптимальный набор метеорологических данных
    [
        'air_temperature', 'dew_temperature', 'sea_level_pressure', 'wind_speed', 'cloud_coverage',
        'air_temperature_diff1', 'air_temperature_diff2'
    ],
    
    2: #во второй - дни недели и праздники
    WEEKDAY_FEATURES + ['is_holiday'],
    
    3: #в третьей - недели года
    WEEK_FEATURES,
    
    4: #в четвертой - месяцы
    MONTH_FEATURES
}

TARGET_FEATURE = 'meter_reading'

<a id='etl'></a>
## 2. ETL
[Вспомогательные функции для очистки и преобразования исходных данных](etl_utils.py)

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
pd.set_option('display.max_columns', 200)
from etl_utils import reduce_mem_usage, show_inf_and_na, inf_and_na_columns


def assemble_the_data(energy_df, weather_df, buildings_df, max_building_number=NUMBER_OF_BUILDINGS_FOR_MODELLING):
    assembeled_df = energy_df[energy_df['building_id'] < max_building_number]
    assembeled_df = pd.merge(left=assembeled_df, right=buildings_df, how='left', left_on='building_id', right_on='building_id')
    assembeled_df = pd.merge(
        left=assembeled_df.set_index(['timestamp', 'site_id']), 
        right=weather_df.set_index(['timestamp', 'site_id']),
        how='left', left_index=True, right_index=True
    )
    assembeled_df.reset_index(inplace=True)
    assembeled_df["precip_depth_1_hr"] = assembeled_df["precip_depth_1_hr"].map(lambda x: x if x > 0 else 0)
    
    for column in inf_and_na_columns(assembeled_df):
        assembeled_df[column] = assembeled_df[column].interpolate(limit_direction='both', kind='cubic')
    assembeled_df = assembeled_df.drop(inf_and_na_columns(assembeled_df), axis='columns')
    return reduce_mem_usage(assembeled_df)


def enrich_weather_data(df: pd.DataFrame):
    df['air_temperature_diff1'] = df['air_temperature'].diff()
    df.at[0, 'air_temperature_diff1'] = df.at[1, 'air_temperature_diff1']
    df['air_temperature_diff2'] = df['air_temperature_diff1'].diff()
    df.at[0, 'air_temperature_diff2'] = df.at[1, 'air_temperature_diff2']
    return  df


def generate_one_hot_column_df(feature_values: pd.Series, columns: list):
    df = pd.get_dummies(feature_values)
    for i in range(df.shape[1], len(columns)): # если one_hot_vector получился короче, чем columns
        df[columns[i]] = 0
    return df.set_axis(columns, axis='columns')


def enrich_date_data(df: pd.DataFrame):
    us_holidays = calendar().holidays(start=df['timestamp'].dt.date.min(), end=df['timestamp'].dt.date.max())
    df['is_holiday'] = df['timestamp'].isin(us_holidays).astype('int8')
    
    df = pd.merge(
        left=df,
        right=generate_one_hot_column_df(df['timestamp'].dt.weekday.astype('int8'), WEEKDAY_FEATURES),
        left_index=True, right_index=True
    )
    df = pd.merge(
        left=df,
        right=generate_one_hot_column_df(df['timestamp'].dt.isocalendar().week.astype('int8'), WEEK_FEATURES),
        left_index=True, right_index=True
    )    
    df = pd.merge(
        left=df,
        right=generate_one_hot_column_df(df['timestamp'].dt.month.astype('int8'), MONTH_FEATURES),
        left_index=True, right_index=True
    )
    return  df

In [3]:
buildings_df = pd.read_csv('http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz')

weather_train_df = pd.read_csv('http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz')
energy_train_df = pd.read_csv('http://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz')

weather_test_df = pd.read_csv('http://video.ittensive.com/machine-learning/ashrae/weather_test.csv.gz')
energy_test_df = pd.read_csv('http://video.ittensive.com/machine-learning/ashrae/test.csv.gz')

train_df = enrich_date_data(enrich_weather_data(assemble_the_data(energy_train_df, weather_train_df, buildings_df)))
train_df = train_df[train_df[TARGET_FEATURE] > 0]
del energy_train_df
del weather_train_df

test_df = enrich_date_data(enrich_weather_data(assemble_the_data(energy_test_df, weather_test_df, buildings_df)))
del energy_test_df
del weather_test_df
del buildings_df

train_df.shape, test_df.shape

Потребление памяти меньше на 35.47 Мб (-71.7%)
Потребление памяти меньше на 84.21 Мб (-70.0%)


((269067, 90), (1051200, 90))

<a id='modelling'></a>
## 3. Построение для каждого типа (набора признаков) моделей линейной регрессии

In [4]:
def models_amount(models):
    amount = 0
    for i in range(len(models)):
        amount += len(models[i])
    return amount


def generate_hourly_models(df: pd.DataFrame, x_columns: list, y_column: str) -> dict:
    '''
    Генерирует набор моделей линейной регрессии.
    Количество моделей соответствует колчеству часов представленных в df['timestamp']
    df['timestamp'] - ОБЯЗАТЕЛЬНЫЙ столбец во фрейме данных
    x_columns - список столбцов, которые используются в качестве входных данных (факторных переменных) при обучении моделей
    y_column - столбец целевой переменной при обучении моделей
    
    Возвращает словарь в котором ключ - это час, а значение - модель для этого часа
    '''
    hourly_models = dict()
    for hour in range(0, df['timestamp'].dt.hour.max() + 1):
        hour_data = df[df['timestamp'].dt.hour == hour]
        if len(hour_data) > 0:
            hourly_models[hour] = LinearRegression().fit(hour_data[x_columns].values, hour_data[y_column].values)
    return hourly_models


ensemble = dict()
for model_type in FEATURE_TYPES:
    ensemble[model_type] = dict()
    for building_id in range(0, train_df['building_id'].max() + 1):
        ensemble[model_type][building_id] = generate_hourly_models(
            df=train_df[train_df['building_id'] == building_id],
            x_columns=FEATURE_TYPES[model_type],
            y_column=TARGET_FEATURE
        )
    print('Тип', model_type, '- всего моделей:', models_amount(ensemble[model_type]))

Тип 1 - всего моделей: 1200
Тип 2 - всего моделей: 1200
Тип 3 - всего моделей: 1200
Тип 4 - всего моделей: 1200


<a id='prediction'></a>
## 4. Построение прогнозов для каждого типа моделей

In [5]:
def make_prediction(row, x_columns, models):
    '''
    Строит прогнозное значение
    row - кортеж входных данных    
    row['building_id'] - ОБЯЗАТЕЛЬНЫЙ атрибут во входном кортеже данных
    row['timestamp'] - ОБЯЗАТЕЛЬНЫЙ атрибут во входном кортеже данных
    
    x_columns - список названий атрибутов из row, по которым нужно построить прогноз
    models - словарь, в котором располагаются модели по последовательности ключей: [идентификатор здания][час] 
    Возвращает число float
    '''
    result = 0
    building_id = row['building_id']
    if building_id in models.keys():
        hour = row['timestamp'].hour
        if hour in models[building_id].keys():
            result = models[building_id][hour].predict(row[x_columns].values.reshape(1, -1))[0]
    
    if result < 0:
        result = 0
    return np.float16(result)

In [6]:
%%time
predictions = dict()
for model_type in FEATURE_TYPES:
    predictions[model_type] = test_df.apply(
        make_prediction,
        axis='columns',
        x_columns=FEATURE_TYPES[model_type],
        models=ensemble[model_type]
    )
    print('Тип', model_type, '- прогнозы построены.')

Тип 1 - прогнозы построены.
Тип 2 - прогнозы построены.
Тип 3 - прогнозы построены.
Тип 4 - прогнозы построены.
CPU times: total: 26min 24s
Wall time: 26min 25s


In [7]:
for model_type in FEATURE_TYPES:
    predictions[model_type].fillna(value=0, inplace=True)

<a id='result'></a>
## 5. Вычисление финального значения как взвешенного арифметического среднего:
* веса для первой и второй модели как 3/8
* третьей и четвертой - как 1/8.

In [8]:
test_df[TARGET_FEATURE] = ((3/8) * predictions[1] + (3/8) * predictions[2] + (1/8) * predictions[3] + (1/8) * predictions[4])
results_df = test_df[['row_id', TARGET_FEATURE]]
results_df.head()

,row_id,meter_reading
0,0,186.000000
1,1,82.000000
2,2,7.945312
3,3,283.000000
4,4,1245.000000


<a id='create_submision'></a>
## 6. Формирование выгрузки данных для тех зданий, которые посчитаны в модели

In [9]:
submission_df = pd.read_csv('http://video.ittensive.com/machine-learning/ashrae/test.csv.gz', usecols=['row_id'])
submission_df = pd.merge(left=submission_df, right=results_df, how='left', left_on='row_id', right_on='row_id')
submission_df = reduce_mem_usage(submission_df.fillna(0))
submission_df.info()

Потребление памяти меньше на 159.06 Мб (-22.2%)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 41697600 entries, 0 to 41697599
Data columns (total 2 columns):
 #   Column         Dtype  
---  ------         -----  
 0   row_id         int32  
 1   meter_reading  float16
dtypes: float16(1), int32(1)
memory usage: 556.7 MB


<a id='save_submision'></a>
## 7. Выгрузка результат в виде CSV-файла (submission.csv)

In [10]:
submission_df.to_csv('submission.csv', index=False)

### Освобождение памяти

In [11]:
del train_df
del test_df
del results_df
del submission_df